In [65]:
from twython import Twython
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm
import re # regular expressions
from tqdm import tnrange, tqdm_notebook, tqdm

In [66]:
# The currency can be changed and the extraction will change depending on these inouts
currency = "bitcoin"
symbol = "BTC"

# Files to write tweets
tweets_raw_file = f'../data/twitter/{symbol}/{currency}_tweets_raw.csv'
tweets_clean_file = f'../data/twitter/{symbol}/{currency}_tweets_clean.csv'

# Query to run on the Twitter API
query = f'#{currency} OR #{symbol}'

In [67]:
# App Key and secret obtained from the application in Twitter developer portal
app_key = '2IGXQ4kdoysrGMeaUZkSvGCvM'
app_secret = 'ZS6drXCpvUlQJhdQ7g3XbgCaOLI5IdPeA6fKwtEGkTIrx5YHaR'

# Declaration of the Twython Object
twitter = Twython(app_key, app_secret, oauth_version = 2)
access_token = twitter.obtain_access_token()
twitter = Twython(app_key, access_token = access_token)
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1641930317}}

In [68]:
# Maximum number of queries every 15 minutes as per Twitter API
# Object declaration for the list of tweets
queries_number = 450
data = {"tweets": []}

# Open tweets file
with open(tweets_raw_file,"a+", encoding='utf-8') as f:
    # Write the column names in the file
    f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n")
    twitter = Twython(app_key, access_token=access_token)
    last_size = 0
    
    # Run first query and declare the next id as the last tweet
    data = twitter.search(q = query, lang = 'en', result_type = 'recent', count = "100") # Use since_id for tweets after id
    next_id = data["tweets"][len(data["tweets"]) - 1]['id']
    
    while(True):
        twitter = Twython(app_key, access_token=access_token)
        last_size = 0
        # Run 450 queries every 15 minutes
        for i in tqdm(range(queries_number - 1)):
            if not next_id:
                data = twitter.search(q = query, lang = 'en', result_type = 'recent', count = "100") # Use since_id for tweets after id
            else:
                data["tweets"].extend(twitter.search(q = query, lang = 'en', result_type = 'mixed', count = "100", max_id = next_id)["tweets"])
            if len(data["tweets"]) > 1:
                next_id = data["tweets"][len(data["tweets"]) - 1]['id']
            if last_size + 1 == len(data["tweets"]):
                break
            else:
                last_size = len(data["tweets"])

        print('Got {0} tweets, wait 15 minutes for next queries as per Twitters API usage policy.'.format(len(data["tweets"])))
        d = pd.DataFrame([[s["id"], s["text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["tweets"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        # Stop if the end is reached
        if last_size + 1 == len(data["tweets"]):
            print('Stop...')
            break
        data["tweets"] = []
        # Sleep 15 minutes between every loop
        sleep(910)

100%|██████████| 449/449 [02:22<00:00,  3.16it/s]


Retrieved 41253, waiting for 15 minutes until next queries


100%|██████████| 449/449 [02:23<00:00,  3.13it/s]


Retrieved 39099, waiting for 15 minutes until next queries


100%|██████████| 449/449 [02:16<00:00,  3.29it/s]


...

Retrieved 39322, waiting for 15 minutes until next queries


100%|██████████| 449/449 [02:13<00:00,  3.37it/s]


Retrieved 40508, waiting for 15 minutes until next queries


 84%|████████▎ | 376/449 [01:48<00:21,  3.47it/s]


Retrieved 33416, waiting for 15 minutes until next queries
No more new tweets, stopping...


In [69]:
# Clean the tweets of unnecesary text for better sentiment analysis
data = pd.read_csv(tweets_raw_file)
for i, s in enumerate(tqdm(d['Text'])):
    text = data.loc[i, 'Text']
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    d.loc[i, 'Text'] = text
file = open(tweets_clean_file, 'a+', encoding='utf-8')
data.to_csv(file, header=True, encoding='utf-8',index=False)

/Users/aguerrero023/opt/anaconda3/envs/pyvizenv2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1933729/1933729 [05:40<00:00, 5671.32it/s]


In [70]:
df_clean = pd.read_csv(tweets_clean_file)
df_clean.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,1480974073383448576,RT : ⚜💎 Premium Diamond 💎⚜🏈 NCAAF 💎✅☑✅☑✅☑✅☑✅💎💎...,JC’S Picks,843,22,0,Tue Jan 11 18:45:08 +0000 2022
1,1480974072938905603,"RT : Honorable Judges, We are requesging to le...",Aman. Panhwar,29,11,0,Tue Jan 11 18:45:08 +0000 2022
2,1480974071894401027,New Airdrop: AMMpad🔘 Details: /Wl71sALlsIAirdr...,gaktau,19,0,0,Tue Jan 11 18:45:08 +0000 2022
3,1480974070447550471,RT : BREAKING – Strike officially launches Bit...,BitCapt,139,322,0,Tue Jan 11 18:45:08 +0000 2022
4,1480974070132924426,RT : Top 10 cryptocurrencies in the world by T...,jeffrockz _ zaid,59,4,0,Tue Jan 11 18:45:08 +0000 2022


In [71]:
df_clean.min(axis=0)

/Users/aguerrero023/opt/anaconda3/envs/pyvizenv2/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Text          3 months ago all newbie were like lions now t...
CreatedAt                                            CreatedAt
dtype: object

In [72]:
file.close()